In [ ]:
import pickle
from forward import *

def display(data):
    pickle.dump(data, open('../FORWARD/interface/data/active.pickle', 'wb'))

In [ ]:
frames_red = np.fromfile('inspect/frames_red.data', dtype=np.uint8)
frames_green = np.fromfile('inspect/frames_green.data', dtype=np.uint8)
frames_blue = np.fromfile('inspect/frames_blue.data', dtype=np.uint8)

frame = np.array([frames_red, frames_green, frames_blue]).T.reshape(240, 256, 3)

plt.axis('off')
plt.imshow(frame);

display([Image(frame), Image(np.ones((1,1000)))])

In [ ]:
(ppu_memory[0x3F00:0x3F1F],
ppu_memory[0x3F20:0x3F2F])

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

def combine(plane1, plane2):
    plane1 = [int(c) for c in f'{plane1:08b}']
    plane2 = [int(c) for c in f'{plane2:08b}']

    return [n2*2 + n1*1 for n2, n1 in zip(plane2, plane1)]

with open('inspect/ppu_memory.data', 'rb') as file:
    ppu_memory = file.read()

with open('inspect/oam.data', 'rb') as file:
    oam_memory = file.read()

ppu_memory = np.fromfile('inspect/ppu_memory.data', dtype=np.uint8)
oam_memory = np.fromfile('inspect/oam.data', dtype=np.uint8)

pattern_table0 = ppu_memory[0x0000:0x1000]
pattern_table1 = ppu_memory[0x1000:0x2000]
nametable0 = ppu_memory[0x2000:0x2000+960]
attributes0 = ppu_memory[0x2000+960:0x2400]
nametable1 = ppu_memory[0x2400:0x2400+960]
nametable2 = ppu_memory[0x2800:0x2C00]
nametable3 = ppu_memory[0x2C00:0x3000]

palette_memory = ppu_memory[0x3F00:0x3F10]
palette_memory

oam_memory.reshape(-1, 4)

In [ ]:
colors = [[84, 84, 84], [0, 30, 116], [8, 16, 144], [48, 0, 136], [68, 0, 100], [92, 0, 48], [84, 4, 0], [60, 24, 0], [32, 42, 0], [8, 58, 0], [0, 64, 0], [0, 60, 0], [0, 50, 60], [0, 0, 0], [0, 0, 0], [0, 0, 0],
[152, 150, 152], [8, 76, 196], [48, 50, 236], [92, 30, 228], [136, 20, 176], [160, 20, 100], [152, 34, 32], [120, 60, 0], [84, 90, 0], [40, 114, 0], [8, 124, 0], [0, 118, 40], [0, 102, 120], [0, 0, 0],[0, 0, 0], [0, 0, 0],
[236, 238, 236], [76, 154, 236], [120, 124, 236], [176, 98, 236], [228, 84, 236], [236, 88, 180], [236, 106, 100], [212, 136, 32], [160, 170, 0], [116, 196, 0], [76, 208, 32], [56, 204, 108], [56, 180, 204], [60, 60, 60],[0, 0, 0], [0, 0, 0],
[236, 238, 236], [168, 204, 236], [188, 188, 236], [212, 178, 236], [236, 174, 236], [236, 174, 212], [236, 180, 176], [228, 196, 144], [204, 210, 120], [180, 222, 120], [168, 226, 144], [152, 226, 180], [160, 214, 228], [160, 162, 160], [0, 0, 0], [0, 0, 0]]

np.array(colors);

In [ ]:
sprites = np.split(pattern_table1, 256)
sprites = [[combine(a, b) for a, b in zip(sprite[:8], sprite[8:])]
           for sprite in sprites]

blocks = [sprites[id] for id in nametable0]

arr = np.concatenate(np.split(np.concatenate(blocks, axis=1), 30, axis=1))

arr2 = arr.copy()
arr2 = np.stack([arr2, np.zeros(arr2.shape), np.zeros(arr2.shape)]).transpose(1, 2, 0)
arr2 = arr2.astype(np.uint8)
# print(arr2.shape)

for i in range(arr2.shape[0]):
    for j in range(arr2.shape[1]):
        palette_index = arr2[i, j, 0]
        code = attributes0.reshape(8, 8)[i // 32][j // 32];
        bit1 = (i // 16) % 2
        bit0 = (i // 16) % 2
        bit_value = bit1*2 + bit0*1
        shift = [0, 2, 4, 6][bit_value]
        palette = (code >> shift) % 4

        # palette = 0

        color_index = palette_memory[palette*4 + palette_index]
        # print(palette, palette_index, color_index)
        color = colors[color_index]
        arr2[i, j] = color

plt.axis('off')
plt.imshow(arr2);

In [ ]:
sprites = np.split(np.concatenate([pattern_table0, pattern_table1]), 512)
sprites = [[combine(a, b) for a, b in zip(sprite[:8], sprite[8:])]
           for sprite in sprites]

arr = np.concatenate(np.split(np.concatenate(sprites, axis=1), 32, axis=1))
arr = np.concatenate(arr.reshape(2, 128, 128), axis=1)

plt.axis('off')
plt.imshow(arr);